In [ ]:
import os
import h5py
import numpy as np
import pandas as pd
import geopandas as gp
from operator import attrgetter
from scipy import stats
from scipy.interpolate import interp1d
from scipy import signal
from math import tan, pi
import scipy.spatial as spatial
from sklearn.preprocessing import StandardScaler

# Signal photons extraction from ATL03 data product

In [ ]:
"""Python code below allows to retrieve the desired data from ICESat-2 ATL03 data product,
   the data are then organized in a dataframe for each ground track"""

import os
import sys
import h5py
import warnings
import numpy as np
import pandas as pd
import geopandas as gpd

               
ATL03_files = ('/data/atmani/ATL03_new/')

for f in os.listdir(ATL08_files):
    if f.startswith('ATL03') and f.endswith('.h5'):
        fid = os.path.join(ATL03_files, f)
        ATL03 = h5py.File(fid,'r')
        
        gtr = [g for g in ATL03.keys() if g.startswith('gt')]
        
        ATL03_objs = []
        ATL03.visit(ATL03_objs.append)                                           
        ATL03_SDS = [o for o in ATL03_objs if isinstance(ATL03[o], h5py.Dataset)]
        
        lat_ph, lon_ph, h_ph, dist_ph_along, signal_conf_ph, gtx = ([] for i in range(6))
        
        # Retrieve datasets
        for b in gtr:
            #for c in ATL03:
            [lat_ph.append(h) for h in ATL03[[g for g in ATL03_SDS if g.endswith('/lat_ph') and b in g][0]][()]]
            [lon_ph.append(h) for h in ATL03[[g for g in ATL03_SDS if g.endswith('/lon_ph') and b in g][0]][()]]
            [h_ph.append(h) for h in ATL03[[g for g in ATL03_SDS if g.endswith('/h_ph') and b in g][0]][()]]   
            [dist_ph_along.append(h) for h in ATL03[[g for g in ATL03_SDS if g.endswith('/dist_ph_along') and b in g][0]][()]]
            [signal_conf_ph.append(h) for h in ATL03[[g for g in ATL03_SDS if g.endswith('/signal_conf_ph') and b in g][0]][()]]
            
            ATL03_df = pd.DataFrame({'Latitude': lat_ph, 'Longitude': lon_ph, 'Along-track_Distance': dist_ph_along,
                                     'Photon_Height': h_ph, 'Signal_Confidence':signal_conf_ph})
            
            del lat_ph, lon_ph, h_ph, dist_ph_along, signal_conf_ph
            
            ATL03_df.loc[:, 'Land'] = ATL03_df.Signal_Confidence.map(lambda x: x[0])
            ATL03_df = ATL03_df.drop(columns=['Signal_Confidence'])
            
            # Transform coordinates into utm
            from pyproj import proj
            from pyproj import Transformer
            x, y = np.array(ATL03_df['Longitude']), np.array(ATL03_df['Latitude'])
            transformer = Transformer.from_crs('epsg:4326', 'epsg:32733', always_xy=True)
            xx, yy = transformer.transform(x, y)
            
            # Save the utm coordinates into the dataframe
            ATL03_df['Easting'] = xx 
            ATL03_df['Northing'] = yy
            
            ATL03_df, rotation_data = get_atl_alongtrack(ATL03_df)

            ROI = gpd.GeoDataFrame.from_file('/data/atmani/study_area/Msc_Study_area.shp', crs='EPSG:4326') 
            
            minLon, minLat, maxLon, maxLat = ROI.envelope[0].bounds
            
            # Subset the dataframe into the study area bounds
            ATL03_df = ATL03_df.where(ATL03_df['Latitude'] > minLat)
            ATL03_df = ATL03_df.where(ATL03_df['Latitude'] < maxLat)
            ATL03_df = ATL03_df.where(ATL03_df['Longitude'] > minLon)
            ATL03_df = ATL03_df.where(ATL03_df['Longitude'] < maxLon)
            ATL03_df = ATL03_df.dropna()
            
            ATL03_df.to_csv('{}_{}.csv'.format(ATL03_files[23:-3], gtr), header=True)

# Ground and preliminary canopy photons classification

In [ ]:
path = "/data/atmani/Sig_Ph_06062021/sub/"

for f in os.listdir(path):
    if f.startswith('Fi') and f.endswith('.csv'):
        fid = os.path.join(path, f)
        ATL03_df = pd.read_csv(fid)
        
        # Bining in the along-track direction (bins of 30 m)
        rows_labels = [f"[{i}, {i+30}])" for i in range(int(min(ATL03_df['alongtrack'])), int(max(ATL03_df['alongtrack'])), 30)]
        rows_bins = pd.IntervalIndex.from_tuples([(i, i+30) for i in range(int(min(ATL03_df['alongtrack'])), int(max(ATL03_df['alongtrack'])), 30)], closed="left")
        rows_binned = pd.cut(ATL03_df['alongtrack'], rows_bins, labels=rows_labels, precision=2, include_lowest=True)
        rows_binned.sort_values(ascending=True, inplace=True)

        rows_binned.drop_duplicates(keep='first',inplace=True)
        rows_left = np.asarray(rows_binned.map(attrgetter('left')))
        rows_right = np.asarray(rows_binned.map(attrgetter('right')))
        rows_left = rows_left[~np.isnan(rows_left)]
        rows_right = rows_right[~np.isnan(rows_right)]

        ground_ph = []
        canop = []
        for i in range(len(rows_left)):
            df = ATL03_df.where((ATL03_df['alongtrack']>= rows_left[i]) & (ATL03_df['alongtrack'] < rows_right[i]))
            df = df.dropna()

            # Topography detrending and outliers filtering
            df['detrend'] = signal.detrend(df['Photon_Height'])
            df = df.where((df['detrend']> -(30*tan((pi/180)*30))) & (df['detrend'] < (30*tan((pi/180)*30))))
            df = df.dropna()
            df = df.drop(columns=['detrend'])
            
            # Retrieving photons between the 25th and the 75th percentiles
            if len(df)>5:
                df_grd = df.where((df['Photon_Height'] <= np.percentile(df['Photon_Height'],75)) & (df['Photon_Height'] > np.percentile(df['Photon_Height'],25))) 
                df_grd = df_grd.dropna()
                
                #Topography detrend and outliers filtering
                df_grd['detrend'] = signal.detrend(df_grd['Photon_Height'])
                df_grd = df_grd.where((df_grd['detrend']> -(tan((pi/180)*30))) & (df_grd['detrend'] < (tan((pi/180)*30))))
                df_grd = df_grd.dropna()

            # Photons with height above the maximum of each 25th-75th bin are retrieved as preliminary photons
                if len(df_grd)>5:
                    df_canop = df.where(df['Photon_Height'] > max(df_grd['Photon_Height']))  
                    df_canop = df_canop.dropna()
                    canop.append(df_canop)
                    ground_ph.append(df_grd)

        # Preliminary canopy photons
        canop = [j for j in canop if len(j)>=0]
        if len(canop)>0:
            Canopy = pd.concat(canop, axis=0)

        latitude, longitude, along,cross, med, north, east = ([] for i in range(7))

        ground_ph = [j for j in ground_ph if len(j)!=0]

        # Final ground photons are the medians of each bin placed in the center of each bin
        for df in ground_ph:
            al = (max(df['alongtrack']) + min(df['alongtrack']))/2
            along.append(al)
            es = (max(df['Easting']) + min(df['Easting']))/2
            east.append(es)
            nd = (max(df['Northing']) + min(df['Northing']))/2
            north.append(nd)
            cr = (max(df['crosstrack']) + min(df['crosstrack']))/2
            cross.append(cr)
            lat = (max(df['Latitude']) + min(df['Latitude']))/2
            latitude.append(lat)
            lon = (max(df['Longitude']) + min(df['Longitude']))/2         
            longitude.append(lon)
            m = np.median(df['Photon_Height'])
            med.append(m)
            
        # Ground photons dataframe
        median_df = pd.DataFrame({'Latitude': latitude, 'Longitude': longitude, 'alongtrack': along, 'crosstrack': cross,
                                  'Easting': east, 'Northing': north, 'Photon_Height':med})
        
        # Saving the final ground photons into csv file
        if len(median_df) >5:
            pathh = "/data/atmani/New_Gr_CH_Classifi_17_09_2021/27_09_2021/"
            median_df.to_csv(os.path.join(pathh,r'Ground_{}.csv'.format(fid[36:-4])), header=True)
    
            # calculate the canopy height by subtracting the ground height of each canopy photon by interpolating the final ground photons
            if len(Canopy) > 5:
                Canopy = Canopy.where((Canopy['alongtrack'] > min(median_df['alongtrack'])) & (Canopy['alongtrack'] < max(median_df['alongtrack'])))
                Canopy = Canopy.dropna()

            if len(Canopy) > 5:
                f = interp1d(median_df['alongtrack'], median_df['Photon_Height'], kind='cubic')

                Canopy['Canopy_Height'] = Canopy['Photon_Height'] - f(Canopy['alongtrack'])
                
                Canopy.to_csv(os.path.join(pathh,r'Pre_Canopy_{}.csv'.format(fid[36:-4])), header=True)

# Canopy photons classification

In [ ]:
path = '/data/atmani/TOC_03_10_2021/'

gt = []
for f in os.listdir(path):
    if f.startswith('Pre') and f.endswith('.csv'):
        fid = os.path.join(path, f)
        Canopy = pd.read_csv(fid)

        Canopy = Canopy.where(Canopy['Canopy_Height']>=3)
        Canopy = Canopy.dropna()

        # Easting, Northing and Canopy Height scaling
        if len(Canopy)>5:
            df = Canopy
            df['z'] = (df['Canopy_Height']-np.min(df['Canopy_Height']))/(np.max(df['Canopy_Height'])-np.min(df['Canopy_Height']))
            df['x'] = (df['alongtrack']-np.min(df['alongtrack']))/(np.max(df['alongtrack'])-np.min(df['alongtrack']))

            X = np.array(df[['x', 'z']])
             T = spatial.cKDTree(X)

            index = []
            for i in range(len(df)):
                idx = T.query_ball_point(X[i], r = 0.01)
                index.append(len(idx))
            df['NN'] = index

            # Filtering out canopy photons with number of neighbors below the 15th percentile of the number of neighbors
            ddf = df.where(df['NN'] >=6)
            ddf = ddf.dropna()

            # Binning the along-track direction (bins of 10 m)
            if len(ddf)>0:
                rows_labels = [f"[{i}, {i+10}])" for i in range(int(min(ddf['alongtrack'])), int(max(ddf['alongtrack'])), 10)]
                rows_bins = pd.IntervalIndex.from_tuples([(i, i+10) for i in range(int(min(ddf['alongtrack'])), 
                                                                                   int(max(ddf['alongtrack'])), 10)], closed="left")

                rows_binned = pd.cut(ddf['alongtrack'], rows_bins, labels=rows_labels, precision=2, include_lowest=True)
                rows_binned.sort_values(ascending=True, inplace=True)

                rows_binned.drop_duplicates(keep='first',inplace=True)
                rows_left = np.asarray(rows_binned.map(attrgetter('left')))
                rows_right = np.asarray(rows_binned.map(attrgetter('right')))
                rows_left = rows_left[~np.isnan(rows_left)]
                rows_right = rows_right[~np.isnan(rows_right)]

                df_canop = []
                for i in range(len(rows_left)):
                    dff = ddf.where((ddf['alongtrack']>= rows_left[i]) & (ddf['alongtrack'] < rows_right[i]))
                    dff = dff.dropna()
                    dff['Photons_Numb'] = len(dff)
                # Calculate the maximum photon height of each bin
                    if len(dff)>0:
                        dd = dff.where(dff['Canopy_Height']==max(dff['Canopy_Height']))
                        dd = dd.dropna()
                        df_canop.append(dd)
                df_canop = [j for j in df_canop if len(j)>=0]
                if len(df_canop)>0:

                    toc = pd.concat(df_canop, axis=0)

                    # Save the canopy and top of canopy photons into new csv files 
                    pathh = '/data/atmani/TOC_03_10_2021/TOC_02_01_2022_6pts_0.1/'
                    toc.to_csv(os.path.join(pathh,r'TOC_{}.csv'.format(toc['gt'].iloc[0])), header=True)

# Grass photons classificaion

In [ ]:
path = '/data/atmani/TOC_03_10_2021/'

gt = []
for f in os.listdir(path):
    if f.startswith('Pre') and f.endswith('.csv'):
        fid = os.path.join(path, f)
        Canopy = pd.read_csv(fid)
        
        # All photons with canopy height between 0.5 m and 3 m
        Canopy = Canopy.where((Canopy['Canopy_Height']>=0.5) & (Canopy['Canopy_Height']<3))
        Canopy = Canopy.dropna()

        # Easting, Northing and Canopy Height scaling
        if len(Canopy)>5:
            df = Canopy
            df['z'] = (df['Canopy_Height']-np.min(df['Canopy_Height']))/(np.max(df['Canopy_Height'])-np.min(df['Canopy_Height']))
            df['x'] = (df['alongtrack']-np.min(df['alongtrack']))/(np.max(df['alongtrack'])-np.min(df['alongtrack']))

            X = np.array(df[['x', 'z']])
            T = spatial.cKDTree(X)

            index = []
            for i in range(len(df)):
                idx = T.query_ball_point(X[i], r = 0.01)
                index.append(len(idx))
            df['NN'] = index

            # Filtering out canopy photons with number of neighbors below the 15th percentile of the number of neighbors
            ddf = df.where(df['NN'] >=6)
            ddf = ddf.dropna()

            # Binning the along-track direction (bins of 10 m)
            if len(ddf)>0:
                rows_labels = [f"[{i}, {i+10}])" for i in range(int(min(ddf['alongtrack'])), int(max(ddf['alongtrack'])), 10)]
                rows_bins = pd.IntervalIndex.from_tuples([(i, i+10) for i in range(int(min(ddf['alongtrack'])), 
                                                                                   int(max(ddf['alongtrack'])), 10)], closed="left")

                rows_binned = pd.cut(ddf['alongtrack'], rows_bins, labels=rows_labels, precision=2, include_lowest=True)
                rows_binned.sort_values(ascending=True, inplace=True)

                rows_binned.drop_duplicates(keep='first',inplace=True)
                rows_left = np.asarray(rows_binned.map(attrgetter('left')))
                rows_right = np.asarray(rows_binned.map(attrgetter('right')))
                rows_left = rows_left[~np.isnan(rows_left)]
                rows_right = rows_right[~np.isnan(rows_right)]

                df_canop = []
                for i in range(len(rows_left)):
                    dff = ddf.where((ddf['alongtrack']>= rows_left[i]) & (ddf['alongtrack'] < rows_right[i]))
                    dff = dff.dropna()
                # Calculate the maximum photon height of each bin
                    if len(dff)>0:
                        dd = dff.where(dff['Canopy_Height']==max(dff['Canopy_Height']))
                        dd = dd.dropna()
                        df_canop.append(dd)
                df_canop = [j for j in df_canop if len(j)>=0]
                if len(df_canop)>0:

                    toc = pd.concat(df_canop, axis=0)

                    # Save the canopy and top of canopy photons into new csv files 
                    pathh = '/data/atmani/TOC_03_10_2021/Grass_02_01_2022_6pts_0.1/'
                    toc.to_csv(os.path.join(pathh,r'Grass_height_{}.csv'.format(toc['gt'].iloc[0])), header=True)

# Along-track distance function

In [ ]:
""" The functions below are used to calculate the along-track distance, the functions are the same used by
    PhoREAL (Photon Research and Engineering Analysis Library) https://github.com/icesat-2UT/PhoREAL"""


def getCoordRotFwd(xIn,yIn,R_mat,xRotPt,yRotPt,desiredAngle):
   
    # Get shape of input X,Y data
    xInShape = np.shape(xIn)
    yInShape = np.shape(yIn)
    
    # If shape of arrays are (N,1), then make them (N,)
    xIn = xIn.ravel()
    yIn = yIn.ravel()
    
    # Suppress warnings that may come from np.polyfit
    if not sys.warnoptions:
        warnings.simplefilter("ignore")
    # endif
    
    # If Rmatrix, xRotPt, and yRotPt are empty, then compute them
    if(len(R_mat)==0 and len(xRotPt)==0 and len(yRotPt)==0):
        
        # Get current angle of linear fit data
        x1 = xIn[0]
        x2 = xIn[-1]
        y1 = yIn[0]
        y2 = yIn[-1]
        # endif
        deltaX = x2 - x1
        deltaY = y2 - y1
        theta = np.arctan2(deltaY,deltaX)
        
        # Get angle to rotate through
        phi = np.radians(desiredAngle) - theta
        
        # Get rotation matrix
        R_mat = np.matrix(np.array([[np.cos(phi), -np.sin(phi)],[np.sin(phi), np.cos(phi)]]))
        
        # Get X,Y rotation points
        xRotPt = x1
        yRotPt = y1
    
    else:
        
        # Get angle to rotate through
        phi = np.arccos(R_mat[0,0])
    
    # endif
    
    # Translate data to X,Y rotation point
    xTranslated = xIn - xRotPt
    yTranslated = yIn - yRotPt
    
    # Convert np array to np matrix
    xTranslated_mat = np.matrix(xTranslated)
    yTranslated_mat = np.matrix(yTranslated)
    
    # Get shape of np X,Y matrices
    (xTranslated_matRows,xTranslated_matCols) = xTranslated_mat.shape
    (yTranslated_matRows,yTranslated_matCols) = yTranslated_mat.shape
    
    # Make X input a row vector
    if(xTranslated_matRows > 1):
        xTranslated_mat = np.transpose(xTranslated_mat)
    #endif
    
    # Make Y input a row vector
    if(yTranslated_matRows > 1):
        yTranslated_mat = np.transpose(yTranslated_mat)
    #endif
    
    # Put X,Y data into separate rows of matrix
    xyTranslated_mat = np.concatenate((xTranslated_mat,yTranslated_mat))
    
    # Compute matrix multiplication to get rotated frame
    measRot_mat = np.matmul(R_mat,xyTranslated_mat)
                            
    # Pull out X,Y rotated data
    xRot_mat = measRot_mat[0,:]
    yRot_mat = measRot_mat[1,:]
    
    # Convert X,Y matrices back to np arrays for output
    xRot = np.array(xRot_mat)
    yRot = np.array(yRot_mat)
    
    # Make X,Y rotated output the same shape as X,Y input
    xRot = np.reshape(xRot,xInShape)
    yRot = np.reshape(yRot,yInShape)
    
    # Reset warnings 
    warnings.resetwarnings()
                   
    # Return outputs
    return xRot, yRot, R_mat, xRotPt, yRotPt, phi

class AtlRotationStruct:
    
    # Define class with designated fields
    def __init__(self, R_mat, xRotPt, yRotPt, desiredAngle, phi):
        
        self.R_mat = R_mat
        self.xRotPt = xRotPt
        self.yRotPt = yRotPt
        self.desiredAngle = desiredAngle
        self.phi = phi
        
# Function to calculate the along-track distance
def get_atl_alongtrack(df):
    easting = np.array(df['Easting'])
    northing = np.array(df['Northing'])
    
    desiredAngle = 90
    crossTrack, alongTrack, R_mat, xRotPt, yRotPt, phi = \
    getCoordRotFwd(easting, northing, [], [], [], desiredAngle)

    df = pd.concat([df,pd.DataFrame(crossTrack, columns=['crosstrack'])],axis=1)
    df = pd.concat([df,pd.DataFrame(alongTrack, columns=['alongtrack'])],axis=1)  

    rotation_data = AtlRotationStruct(R_mat, xRotPt, yRotPt, desiredAngle, phi)
    
    return df, rotation_data
